## WORK IN PROGRESS:

##Overview:

This project was inspired by seeing the difficulties experienced by small business owners like my father. The goal was to go beyond the mere observation that the Covid-19 pandemic created economic hardship: what was the relationship between Covid's spread and business difficulties? Did this relationship vary between states? Did it change over time? 
This project aims to begin answering these questions, with the hope of better equipping small business owners with the tools to react.

### Data Details: 
Economic and Covid Data is sourced primarily from Opportunity Insights Economic tracker, a database set up by Harvard researchers. I will briefly overview the data contents, but for more detail see their github and paper:

"The Economic Impacts of COVID-19: Evidence from a New Public Database Built Using Private Sector Data", by Raj Chetty, John Friedman, Nathaniel Hendren, Michael Stepner, and the Opportunity Insights Team. November 2020. Available at: https://opportunityinsights.org/wp-content/uploads/2020/05/tracker_paper.pdf

Currently, the 4 main tables/datasets I am importing are:
* StateID table, for tracking identity of region referred to in each state level dataset
* Small Business Revenue
*Consumer spending

Data before March 13,2020 (the date of the US declaration of national emergency over the novel coronavirus) was omitted. For further details see: https://www.whitehouse.gov/briefing-room/presidential-actions/2021/02/24/notice-on-the-continuation-of-the-national-emergency-concerning-the-coronavirus-disease-2019-covid-19-pandemic/



Further links:
*   https://github.com/OpportunityInsights/EconomicTracker/blob/main/docs/oi_tracker_data_dictionary.md
*   https://github.com/OpportunityInsights/EconomicTracker/blob/main/docs/oi_tracker_data_documentation.md
*   https://github.com/OpportunityInsights/EconomicTracker/tree/main/data




# Set up environment
Declare functions and import library

In [2]:
#import libraries
import pandas as pd
import numpy as np
import scipy as sc
import requests
import io
import seaborn as sns
import matplotlib.pyplot as plt
from pprint import pprint
import math

from sklearn import datasets, linear_model, preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import datetime as dt

%matplotlib inline

In [3]:
## DATA CLEANING FUNCTION STORAGE
def add_state_col(df, stateID_df):
  #extract and transform vector of state ID for each column
  state_id_by_day = list(df['statefips'].array)
  state_list_comp_df = [(state_IDs['statename'][state_IDs['statefips'] == key].array[0]) for index,key in enumerate(state_id_by_day)]
  df['state']= state_list_comp_df

def slice_dict(input_dict, start_indx, end_indx):
  dict_slice = [input_dict[key] for key in list(input_dict.keys())[start_indx:end_indx]]
  return dict_slice

  ## DATA CLEANING FUNCTIONS
def replace_nonnumeric_entries_df_force_nan(test_df, protected_cols):
  mutable_cols = [i for i in test_df.columns.array if i not in protected_cols]
  for index, col in enumerate(test_df.loc[:,mutable_cols].columns):
    test_df.loc[:,col] = pd.to_numeric(test_df.loc[:,col], errors = 'coerce') #force column to become numeric, returning nan if not 
  test_df = test_df.fillna(value = 0) #fill any NA values with 0 
  return test_df

def convert_obj_col_to_numeric(df): #, new_type):
  for col in df.columns:
    if df[col].dtype == object:
      df[col] = df[col].apply(pd.to_numeric, errors = 'coerce') #stack overflow version
      #original version: df[col] = df[col].astype(new_type)
      #print(df[col].dtype) #for DEBUG
  return df

def find_mutable_cols(df, protected_cols):
  mut_cols = [j for j in list(df.columns) if j not in protected_cols]
  return mut_cols

def trim_pre_covid(df, date_covid_start):
  #date_covid_start is date that national emergency was declared 
  pass

In [4]:
#set what plot type to use 
#https://matplotlib.org/stable/gallery/style_sheets/style_sheets_reference.html
plt.style.use('seaborn-deep')

# Import Data and evaluate format

In [5]:
#Declare URL for important opportunity tracker data 
raw_covid_rate_by_state_url = 'https://raw.githubusercontent.com/OpportunityInsights/EconomicTracker/main/data/COVID%20-%20State%20-%20Daily.csv' #current to 2022-01-31
state_IDs_url = 'https://raw.githubusercontent.com/OpportunityInsights/EconomicTracker/main/data/GeoIDs%20-%20State.csv' #IDs used for each state in later tables
spending_by_state_url = 'https://raw.githubusercontent.com/OpportunityInsights/EconomicTracker/main/data/Affinity%20-%20State%20-%20Daily.csv' #affinity is personal spending #ends 2021-11-24
revenue_by_state_url = 'https://raw.githubusercontent.com/OpportunityInsights/EconomicTracker/main/data/Womply%20-%20State%20-%20Daily.csv' #womply is small busines revenue #ends 2021-07-09

In [6]:
#import OT data
covid_rate_per_state = pd.read_csv(raw_covid_rate_by_state_url, dtype = 'str')
state_IDs = pd.read_csv(state_IDs_url, dtype = 'str')
spending_per_state = pd.read_csv(spending_by_state_url, dtype = 'str')
revenue_per_state = pd.read_csv(revenue_by_state_url, dtype = 'str')

Do a preliminary check on the data, inspect it for inconsistencies/mission data:

In [7]:
spending_per_state.head(1).append(spending_per_state.tail(1))

,year,month,day,statefips,freq,spend_all,spend_aap,spend_acf,spend_aer,spend_apg,spend_durables,spend_nondurables,spend_grf,spend_gen,spend_hic,spend_hcs,spend_inpersonmisc,spend_remoteservices,spend_sgh,spend_tws,spend_retail_w_grocery,spend_retail_no_grocery,spend_all_incmiddle,spend_all_q1,spend_all_q2,spend_all_q3,spend_all_q4,provisional
0,2020,1,1,1,d,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,0
34577,2021,11,14,56,w,.212,.149,.,.2,.508,.304,.202,.0895,.92,.294,.,.234,.174,.567,.2,.247,.344,.194,.19,.236,.162,.304,1


In [8]:
covid_rate_per_state.head(1).append(covid_rate_per_state.tail(1))

,year,month,day,statefips,new_case_count,new_death_count,case_count,death_count,vaccine_count,fullvaccine_count,booster_first_count,new_vaccine_count,new_fullvaccine_count,new_booster_first_count,new_test_count,test_count,hospitalized_count,new_case_rate,case_rate,new_death_rate,death_rate,new_test_rate,test_rate,new_vaccine_rate,vaccine_rate,new_fullvaccine_rate,fullvaccine_rate,new_booster_first_rate,booster_first_rate,hospitalized_rate
0,2020,1,7,15,.,.,.,.,.,.,.,.,.,.,.,.,0,.,.,.,.,.,.,.,.,.,.,.,.,0
38741,2022,2,16,56,266,3,152863,1673,334331,291386,120166,238,256,314,1480,1338925,99,45.9,26412,.543,289,256,231344,.0411,57.8,.0442,50.3,.0543,20.8,17


In [9]:
state_IDs.tail(1)

,statefips,statename,stateabbrev,state_pop2019
50,56,Wyoming,WY,578759


In [10]:
revenue_per_state.head(2)

,year,month,day,statefips,revenue_all,revenue_inchigh,revenue_inclow,revenue_incmiddle,revenue_ss40,revenue_ss60,revenue_ss65,revenue_ss70,revenue_food_accommodation,revenue_retail,merchants_all,merchants_inchigh,merchants_inclow,merchants_incmiddle,merchants_ss40,merchants_ss60,merchants_ss65,merchants_ss70,merchants_food_accommodation,merchants_retail
0,2020,1,10,1,-.00967,.0204,-.0179,-.00901,-.0102,.0172,-.0147,.0109,.0233,.000606,.00264,.00874,.0058,-.00249,.0075,.0182,-.0134,.00308,.00672,.00511
1,2020,1,10,2,.0168,-.00106,-.0736,.0384,.0362,-.0213,-.121,.0207,.0123,.0321,.00597,-.000315,.102,.00518,.0369,-.0471,-.041,.0109,.00395,.0142


Looks like the start of both covid_rate_per_state and econ_per_state have missing values, presumably from the fact that they are from periods before COVID-19 was detected/known about. I thus in the next section clean that data, and then set the dataframes to begin after national state of emergency was declared in 2020.

# Data cleaning:
> Removed non numeric values programmatically from DFs of interest



In [11]:
protected_cols = list(covid_rate_per_state.keys())[0:4] + ['state']
#add which state each row is taken from
add_state_col(spending_per_state, state_IDs)
add_state_col(covid_rate_per_state, state_IDs)
add_state_col(revenue_per_state, state_IDs)

revenue_per_state = replace_nonnumeric_entries_df_force_nan(revenue_per_state, protected_cols)
spending_per_state= replace_nonnumeric_entries_df_force_nan(spending_per_state, protected_cols)
covid_rate_per_state = replace_nonnumeric_entries_df_force_nan(covid_rate_per_state, protected_cols)

#drop columns after X period 

march_indx = [] #row in each dataframe that is equal to march 14

In [12]:
spending_per_state.head(1).append(spending_per_state.tail(1))

,year,month,day,statefips,freq,spend_all,spend_aap,spend_acf,spend_aer,spend_apg,spend_durables,spend_nondurables,spend_grf,spend_gen,spend_hic,spend_hcs,spend_inpersonmisc,spend_remoteservices,spend_sgh,spend_tws,spend_retail_w_grocery,spend_retail_no_grocery,spend_all_incmiddle,spend_all_q1,spend_all_q2,spend_all_q3,spend_all_q4,provisional,state
0,2020,1,1,1,0.0,0.000,0.000,0.0,0.0,0.000,0.000,0.000,0.0000,0.00,0.000,0.0,0.000,0.000,0.000,0.0,0.000,0.000,0.000,0.00,0.000,0.000,0.000,0,Alabama
34577,2021,11,14,56,0.0,0.212,0.149,0.0,0.2,0.508,0.304,0.202,0.0895,0.92,0.294,0.0,0.234,0.174,0.567,0.2,0.247,0.344,0.194,0.19,0.236,0.162,0.304,1,Wyoming


In [13]:
covid_rate_per_state.head(1).append(covid_rate_per_state.tail(1))

,year,month,day,statefips,new_case_count,new_death_count,case_count,death_count,vaccine_count,fullvaccine_count,booster_first_count,new_vaccine_count,new_fullvaccine_count,new_booster_first_count,new_test_count,test_count,hospitalized_count,new_case_rate,case_rate,new_death_rate,death_rate,new_test_rate,test_rate,new_vaccine_rate,vaccine_rate,new_fullvaccine_rate,fullvaccine_rate,new_booster_first_rate,booster_first_rate,hospitalized_rate,state
0,2020,1,7,15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0000,0.0,0.0000,0.0,0.0000,0.0,0.0,Hawaii
38741,2022,2,16,56,266.0,3.0,152863.0,1673.0,334331.0,291386.0,120166.0,238.0,256.0,314.0,1480.0,1338925.0,99.0,45.9,26412.0,0.543,289.0,256.0,231344.0,0.0411,57.8,0.0442,50.3,0.0543,20.8,17.0,Wyoming


In [14]:
revenue_per_state.head(1).append(revenue_per_state.tail(1))

,year,month,day,statefips,revenue_all,revenue_inchigh,revenue_inclow,revenue_incmiddle,revenue_ss40,revenue_ss60,revenue_ss65,revenue_ss70,revenue_food_accommodation,revenue_retail,merchants_all,merchants_inchigh,merchants_inclow,merchants_incmiddle,merchants_ss40,merchants_ss60,merchants_ss65,merchants_ss70,merchants_food_accommodation,merchants_retail,state
0,2020,1,10,1,-0.00967,0.0204,-0.0179,-0.00901,-0.0102,0.0172,-0.0147,0.0109,0.0233,0.000606,0.00264,0.00874,0.0058,-0.00249,0.0075,0.0182,-0.0134,0.00308,0.00672,0.00511,Alabama
26587,2021,6,30,55,-0.36700,-0.3500,-0.3150,-0.38500,-0.4050,-0.1410,-0.2250,-0.5280,-0.5640,-0.407000,-0.44200,-0.44600,-0.4320,-0.44400,-0.4360,-0.2910,-0.3150,-0.58200,-0.59000,-0.45200,Wisconsin


Manual inspection of data shows that cleaning has been successful. I thus continue to perform preliminary EDA

In [15]:
cols_of_interest = dict() #create dict where key is information type, and value is the list of columns to use that aren't indexs
cols_of_interest['covid'] = find_mutable_cols(covid_rate_per_state, protected_cols)
cols_of_interest['spending'] =  find_mutable_cols(spending_per_state, protected_cols)
cols_of_interest['revenue'] =   find_mutable_cols(spending_per_state, protected_cols)

# MODEL SELECTION- comparing model performance


## Simplest predictive model: Ordinary Least Squares Linear Regression with 2 input variables (new Covid-19 case rate and new Covid-19 death rate)



In [24]:
## MAIN SCRIPT THAT IS CALLED TO PERFORM LINEAR REGRESSION
def process_run_lin_reg(train_X_df,train_Y_df, test_X_df, test_Y_df, indep_vars, Y_variable):
  #TO: given a Test Data/Labels and Train Data/Labels, run linear regression on chosen values
  
  #transform DF into numpy array: Training set
  train_X_array, train_Y_var = prepare_data_linReg(train_X_df,train_Y_df, indep_vars, Y_variable)
  #transform DF into numpy array: Test set
  test_X_array, test_Y_var = prepare_data_linReg(test_X_df,test_Y_df, indep_vars, Y_variable)

  #run linear regression with numpy arrays of train and test sets
  linreg_r_squared, Y_full_prediction, residuals = run_linear_regression(train_X_array, train_Y_var,test_X_array, test_Y_var, indep_vars, Y_variable)

  return linreg_r_squared, Y_full_prediction, residuals
  #takes 2 DFs as input 

## LINEAR REGRESSION SUB FUNCTIONS
  #RUNS FIRST
def prepare_data_linReg(X_data,Y_data, indep_vars, Y_variable):
  #TO: given 2 data frames, and list of names to use, create 2 numpy arrays (for independent and predicted variable)
  #X_variable is list of keys for X_data df
  df_lengths = sorted([len(X_data), len(Y_data)]) #find which df is shorter 
  truncate_len = df_lengths[0]-1 
  Y_var = np.array(Y_data.loc[:truncate_len,Y_variable].copy())
  X_array = np.array(X_data.loc[:truncate_len,indep_vars].copy()) #rows are days, columns are features
  return X_array, Y_var

  #RUNS SECOND


def run_linear_regression(train_X, train_Y, test_X, test_Y,indep_vars, Y_variable):
  ## MAIN MODEL RUN HERE 
  reg = LinearRegression().fit(train_X,train_Y)
  Y_test_prediction = reg.predict(test_X)
  linreg_r_squared =  round(reg.score(test_X,test_Y),3) #calculates the r2
  #print(linreg_r_squared)
  Y_full_prediction = reg.predict(X_array) #this is for the entire dataset, to be used for the plots
  residuals = test_Y- Y_test_prediction
  ## MODEL OUTPUTS R2, full prediction values,and residuals of prediction
  return linreg_r_squared, Y_full_prediction, residuals


In [18]:

def plot_linear_regression_on_state(Y_prediction, Y_var, state):
#INPUT: COVID is slice df of renue
  #revenue is econ info over time
  #revenue = create_rev_df(stateID_df, stateSBrevenue_df, state)
  #train on X and then test
  plt.figure(figsize=(18,18));
  fig, ax = plt.subplots();
  ax.plot(Y_prediction, color = 'r', label = 'Predicted SB Revenue');
  #plot with dates
  #ax.plot_date(mp_dates, Y_prediction, xdate = True, color = 'r', label = 'Predicted SB Revenue');
  ax.plot(Y_var, label = 'Real SB Revenue');
  plt.legend();
  plt.title(state + ' Linear Regression: Predict SB Revenue with covid spread. R^2 = ' + str(round(reg.score(X_array,Y_var), 2)));
  plt.xlabel('Days since National Emergency Declared');
  plt.ylabel(' Proportional Change in SB Revenue');
  return fig, ax


Here I'm running simple linear regression, predicting **daily consumer spending** in each state, using each state's ** New Covid-19 Case Rate and New Covid-19 Death Rate **. 

The model is trained on every other day of data recorded, and tested on the remaining days. Performance is evaluated using R^2, which is a measure of the % of total variance in the variable of interest explainable by the modeled linear relationship between it and the independent variables. Essentially, it asks how much the independent variables appear to contribute to the dynamics of the dependent variables.

First, though, I need to test the 4 assumptions required for Linear Regression:

*  Homoscedacisity
*  Independence
*  Normality
*  Linearity



In [19]:
#assumption 1: Homoscedacisity


In [22]:

#assumption 2: independence


#assumption 4: linearity



In [ ]:
#test out new prediction code on california, train on 2020, test on 2021
predict_california = []
residual_california = []
r2_california = []
indep_vars = ['new_death_rate','new_case_rate']
Y_variable = ['spend_all']


test_df_x = covid_rate_per_state.loc[covid_rate_per_state.loc[:, 'state']== state, :].copy().reset_index(drop = True)
covid = 


spending = spending_per_state.loc[spending_per_state['state']== state, :].copy().reset_index(drop = True)
r2_california, predict_california, residual_california = process_run_lin_reg(covid,spending, indep_vars, Y_variable) #requires test set and train set


In [23]:
#run forloop to run and store
all_state_names = list(set(state_IDs.loc[:,'statename'])) #use set to extract unique values then combine into a list
predictions_OLS = dict()# store state:Y prediction list
residuals_OLS = dict() #store it for assumption testing

r_2_linreg = list()
indep_vars = ['new_death_rate','new_case_rate']
Y_variable = ['spend_all']

for state in all_state_names:

  covid = covid_rate_per_state.loc[covid_rate_per_state.loc[:, 'state']== state, :].copy().reset_index(drop = True)
  spending = spending_per_state.loc[spending_per_state['state']== state, :].copy().reset_index(drop = True)
  linreg_r_squared, predictions_OLS[state], residuals_OLS[state] = process_run_lin_reg(covid,spending, indep_vars, Y_variable)
  r_2_linreg.append(linreg_r_squared)



NameError: ignored

In [ ]:
#combine all of this into a table
data = np.array([all_state_names, r_2_linreg]).transpose()
df_model_performance_by_state = pd.DataFrame(data, columns = ['State Name', 'OLSR R2'])
df_model_performance_by_state.loc[:, 'OLSR R2'] =  df_model_performance_by_state.loc[:, 'OLSR R2'].astype('float')

In [ ]:
#assumption 3: normality
#Testing for normality of residuals
import statsmodels.api as sm
fig, ax = plt.subplots()
plt.plot(predictions_OLS['California']);


In [ ]:

plt.plot(spending_per_state.loc[spending_per_state['state'] == 'California', 'spend_all'])


In [ ]:

sm.qqplot(residuals_OLS['California'])

In [ ]:
df_model_performance_by_state.head()

Above is a slice of the table I created, listing 50 states and D.C. and the strength of the relationship between Covid Spread and personal Credit Card Spending from 2020-late 2021. As you can see, the range of R2 values varies widely, which is surprising. 

My Hypothesis was that the relationship between spending on a state level would be relatively similar across states. In other words, either the relationship between changes in Covid-19 rates, and changes in spending would be linearly related in a similar fashion across states, or that the relationship would be non-linear in a similar fashion across states. Instead, it appears that the relationship is linear in *some* states, but not others, which is unexpected. I further investigate potential reasons why.

In [ ]:
fig, axs =plt.subplots(1,2, figsize = (12,6))
# display distribution
sns.lineplot(x = df_model_performance_by_state.loc[:,'OLSR R2'].index,y =df_model_performance_by_state.loc[:,'OLSR R2'].values, ax = axs[0]);#, x = 'R2', binwidth = 5);
axs[0].set_title('Lineplot of Each State R2 value for OLSR');
sns.histplot(x = df_model_performance_by_state.loc[:,'OLSR R2'].values, ax = axs[1]);
axs[1].set_title('Distribution of R2 Values for OLSR');

## Comparing OLSR to Lasso Regression

It could be the case that additional factors besides case counts affect consumer spending. For example, spending rates might increase with vaccination rates, despite decreasing by some amount with case rates. To include more potentially informative variables while attempting to avoid issues with introducing redundant factors/colinearity, I will use Lasso regression, which introduces an L1 regularization term to the Cost Function, ideally driving coefficients for non-essential variables to zero.

I then compare the R^2 value of lasso regression on the full dataset, with ordinary least squares linear regression on the full dataset.


In [ ]:
def normalize_array(X_array, norm_method):
  #this assumes INPUT ARRAY is columns as features, rows as observations
  X_array_norm = np.empty(X_array.shape)
  if norm_method == 'none':
        pass
        #perform no normalization
  elif norm_method == 'zscore':
        #perform z scoring on the data
        X_array_norm[:,:] = sc.stats.zscore(X_array, axis = 0)

  elif norm_method == 'normalize':
        for i in np.arange(X_array.shape[1]):
          X_array_norm[:,i] = (X_array[:,i]- np.min(X_array[:,i]))/(np.max(X_array[:,i])- np.min(X_array[:,i]))

  return X_array_norm

def process_run_lasso_reg(X_data,Y_data, indep_vars, Y_variable, run_cross_val, norm_method):
    X_array, Y_var = prepare_data_linReg(X_data,Y_data, indep_vars, Y_variable)
    #switch case for normalization method
    
    lasso_model, lasso_reg_r_squared, Y_full_prediction, residuals= run_lasso_regression(X_array, Y_var, run_cross_val)

    return lasso_model, lasso_reg_r_squared, Y_full_prediction, residuals

def create_every_other_elem_split(X_array, Y_var):
  train_X =  X_array[0:-1:2,:] #take every other element (step size 2 from 0)
  test_X =  X_array[1::2,:] #take every other element (step size 2 from 1)
  train_Y = Y_var[0:-1:2] #take every other element (step size 2 from 0)
  test_Y = Y_var[1::2] #take every other element (step size 2 from 1)
  return train_X, test_X, train_Y, test_Y

def run_lasso_regression(X_array, Y_var,run_cross_val):
  #create test train split by taking random 50%
  train_X, test_X, train_Y, test_Y= create_every_other_elem_split(X_array, Y_var)

  if run_cross_val:
    lasso_reg = linear_model.LassoCV(cv = 10, tol=0.01, max_iter = 2000) #Cross-val- instantiate lasso class 
    lasso_reg_model = lasso_reg.fit(train_X,train_Y.ravel())
    Y_test_prediction = lasso_reg.predict(test_X)
    lasso_reg_r_squared =  round(lasso_reg.score(test_X,test_Y.ravel()),3) #calculates the r2

  else:
    lasso_reg = linear_model.Lasso(alpha = 0.1) #instantiate lasso class 
    lasso_reg_model = lasso_reg.fit(train_X,train_Y)
    Y_test_prediction = lasso_reg.predict(test_X)
    lasso_reg_r_squared =  round(lasso_reg.score(test_X,test_Y),3) #calculates the r2

  residuals = test_Y - Y_test_prediction
  Y_full_prediction = lasso_reg.predict(X_array) #this is for the entire dataset, to be used for the plots
  return lasso_reg, lasso_reg_r_squared, Y_full_prediction, residuals 


In [ ]:
#cross validate params on one dataset to determine appropriate alphs
indep_vars = cols_of_interest['covid'] #expanding breadth of variables used 
Y_variable = ['spend_all']
run_cross_val = True
#create storage variables for model and predictions
r_2_lasso = list()
predictions_lasso = dict()
residuals_lasso = dict()
lasso_model_by_state = dict()

# create cutoff for date input

for state in all_state_names:

  covid = covid_rate_per_state.loc[covid_rate_per_state.loc[:, 'state']== state, :].copy().reset_index(drop = True)
  spending = spending_per_state.loc[spending_per_state['state']== state, :].copy().reset_index(drop = True)

  lasso_model_by_state[state], lasso_reg_r_squared, predictions_lasso[state], residuals_lasso[state] = process_run_lasso_reg(covid,spending, indep_vars, Y_variable,run_cross_val)
  r_2_lasso.append(lasso_reg_r_squared)

df_model_performance_by_state.loc[:, 'LASSO R2'] = r_2_lasso

In [ ]:
# plot performance
fig, axs =plt.subplots(1,2, figsize = (12,6))
# display distribution
sns.lineplot(x = df_model_performance_by_state.loc[:,'LASSO R2'].index,y =df_model_performance_by_state.loc[:,'LASSO R2'].values, ax = axs[0]);#, x = 'R2', binwidth = 5);
axs[0].set_title('Lineplot of Each State R2 value for Lasso');
sns.histplot(x = df_model_performance_by_state.loc[:,'LASSO R2'].values, ax = axs[1]);
axs[1].set_title('Distribution of R2 Values for Lasso Reg.');


Lasso is clearly better at predicting values using covid spread. The question is, is it using the variables 'correctly'? 


To answer this, we dive into the parameters used by lasso regression, extracting importance by state and plotting that matrix:

In [ ]:
len(all_state_names)

In [ ]:
lasso_coeffs_array = np.empty((len(all_state_names),lasso_model_by_state[state].coef_.shape[0]))
for i, state in enumerate(all_state_names):
  coeffs = lasso_model_by_state[state].coef_
  lasso_coeffs_array[i, :] = coeffs
lasso_coeffs_array.shape  

In [ ]:

lasso_feature_coeff_df = pd.DataFrame(lasso_coeffs_array, columns = indep_vars, index = all_state_names)
lasso_feature_coeff_df.head()

In [ ]:
df_model_performance_by_state['LASSO-OLSR R2 Delta'] =  df_model_performance_by_state.loc[:,'LASSO R2'].values -  df_model_performance_by_state.loc[:,'OLSR R2'].values

#import CSV of all states
states_by_region = 'https://raw.githubusercontent.com/cphalpert/census-regions/master/us%20census%20bureau%20regions%20and%20divisions.csv'
allstates_df = pd.read_csv(states_by_region)
allstates_df= allstates_df.drop(allstates_df.index[7]).copy()
allstates_df.head()

df_model_performance_by_state = pd.merge(df_model_performance_by_state, allstates_df, left_on = 'State Name', right_on = 'State').drop(axis = 1, labels = ('State Name'))
df_model_performance_by_state.head(2)

In [ ]:

#create boxplot of state values by region
fig, ax = plt.subplots(1,2, figsize = (15,7))
g= sns.stripplot(x = 'Region', y = "LASSO R2", data = df_model_performance_by_state, size = 8, ax = ax[0]);
ax[0].tick_params(labelsize=15)
ax[0].xaxis.label.set_fontsize(20)
ax[0].yaxis.label.set_fontsize(20)
ax[0].set_title('R2 distrib. by Region- Lasso (all params)', fontsize=15);

g= sns.stripplot(x = 'Region', y = "OLSR R2", data = df_model_performance_by_state, size = 8, ax = ax[1]);
ax[1].tick_params(labelsize=15)
ax[1].xaxis.label.set_fontsize(20)
ax[1].yaxis.label.set_fontsize(20)
ax[1].set_title('R2 distrib. by Region- OLSR', fontsize=15);

In [ ]:
## TO DO:
#open parameters to evaluate what lasso promoted
#certain features increase monotonically




Further analysis will attempt to understand if temporal factors affect the relationship between these variables. I will partition the dataset into distinct phases: pre-vaccine, post-vaccine, post-Delta. It may be the case that consumer attitudes shift as a result of weariness. I will also incorporate polling data surrounding "covid fatigue"



## Decision Tree Regression
